In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_20942/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/12 04:05:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/12 04:05:11 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data for the Employee table
data = {
    'Id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17],
    'Company': ['A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'C'],
    'Salary': [2341, 341, 15, 15314, 451, 513, 15, 13, 1154, 1345, 1221, 234, 2345, 2645, 2645, 2652, 65]
}

# Creating a pandas DataFrame
df = pd.DataFrame(data)

# Displaying the DataFrame
print(df)



df_person = spark.createDataFrame(df)
df_person.createOrReplaceTempView("Employee")



    Id Company  Salary
0    1       A    2341
1    2       A     341
2    3       A      15
3    4       A   15314
4    5       A     451
5    6       A     513
6    7       B      15
7    8       B      13
8    9       B    1154
9   10       B    1345
10  11       B    1221
11  12       B     234
12  13       C    2345
13  14       C    2645
14  15       C    2645
15  16       C    2652
16  17       C      65


In [25]:
query = """
    select
        *
    from (
        select
            Id,
            Company,
            Salary,
            rank_salary,
            max(rank_salary) over (partition by company) as max_rank_salary
        from (
                select
                    Id,
                    Company,
                    Salary,
                    row_number() over (partition by company order by salary asc, id asc) as rank_salary
                from Employee e 
            )
        )
    where 
        case 
            when max_rank_salary % 2 = 0 then rank_salary = (max_rank_salary / 2) or rank_salary = ((max_rank_salary / 2) + 1)
            else rank_salary = round(max_rank_salary / 2, 0)
        end
"""

In [22]:
query_1 = """
select
            Id,
            Company,
            Salary,
            rank_salary,
            max(rank_salary) over (partition by company) as max_rank_salary
        from (
                select
                    Id,
                    Company,
                    Salary,
                    row_number() over (partition by company order by salary asc, id asc) as rank_salary
                from Employee e 
            )
        

"""

In [23]:
spark.sql(query_1).show()

+---+-------+------+-----------+---------------+
| Id|Company|Salary|rank_salary|max_rank_salary|
+---+-------+------+-----------+---------------+
|  3|      A|    15|          1|              6|
|  2|      A|   341|          2|              6|
|  5|      A|   451|          3|              6|
|  6|      A|   513|          4|              6|
|  1|      A|  2341|          5|              6|
|  4|      A| 15314|          6|              6|
|  8|      B|    13|          1|              6|
|  7|      B|    15|          2|              6|
| 12|      B|   234|          3|              6|
|  9|      B|  1154|          4|              6|
| 11|      B|  1221|          5|              6|
| 10|      B|  1345|          6|              6|
| 17|      C|    65|          1|              5|
| 13|      C|  2345|          2|              5|
| 14|      C|  2645|          3|              5|
| 15|      C|  2645|          4|              5|
| 16|      C|  2652|          5|              5|
+---+-------+------+

In [26]:
spark.sql(query).show()

+---+-------+------+-----------+---------------+
| Id|Company|Salary|rank_salary|max_rank_salary|
+---+-------+------+-----------+---------------+
|  5|      A|   451|          3|              6|
|  6|      A|   513|          4|              6|
| 12|      B|   234|          3|              6|
|  9|      B|  1154|          4|              6|
| 14|      C|  2645|          3|              5|
+---+-------+------+-----------+---------------+

